<a href="https://colab.research.google.com/github/elizathornton/WiBD-Disaster-Tweets-Project/blob/main/Copy_of_Eliza_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importing all the neccessary libraries
import pandas as pd
import numpy as np #for linear algebra


#Libraries for visualisation
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

#Libraries for formattting and handling text 
import string
import re

#Library for nltk
import nltk 
import nltk.corpus
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud

from collections import defaultdict
from collections import  Counter
stop=set(stopwords.words('english'))

#Library for NN
!pip install transformers
import tensorflow as tf

from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from keras.initializers import Constant
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.cuda.amp import autocast, GradScaler

from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from transformers import DistilBertTokenizerFast, TFDistilBertForSequenceClassification
from transformers import DistilBertForSequenceClassification

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, Dropout, Conv1D, GlobalMaxPooling1D, Flatten


import sklearn
#Library for Splitting Dataset
from sklearn.model_selection import train_test_split
#Library for TFIDF
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
#Library for evaluation
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,classification_report
from sklearn.metrics import precision_recall_fscore_support


import warnings
warnings.filterwarnings("ignore")

import plotly
plotly.offline.init_notebook_mode (connected = True)

!pip install neattext
import neattext as nt
from neattext.pipeline import TextPipeline
from neattext.functions import  remove_emails, fix_contractions, remove_numbers,remove_emojis,remove_special_characters, remove_stopwords, remove_html_tags, remove_urls, remove_puncts,remove_custom_pattern


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 24.4 MB/s eta 0:00:00


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.7/114.7 KB 3.7 MB/s eta 0:00:00


In [ ]:

url = 'https://raw.githubusercontent.com/b3ccaMR/NLP-Disaster-Tweets/main/train.csv'
df = pd.read_csv(url)

#filling in NA's for keyword and location
df['keyword'].fillna('no_keyword', inplace=True)
df['location'].fillna('no_location', inplace=True)


In [ ]:

def clean_data(text):

    text = text.lower() # make it all lower case
    text = re.sub(r'\n', '', text) # take out any line breaks
    
    clean_pipeline = TextPipeline(steps=[remove_numbers, fix_contractions, 
                                            remove_stopwords, remove_emails,remove_emojis, 
                                            remove_html_tags, remove_urls, remove_special_characters])

    text = clean_pipeline.transform(text)
 
    lemmatizer = WordNetLemmatizer()
    text = " ".join([lemmatizer.lemmatize(word) for word in text.split()])

    return text


# apply the "clean" function to the data frame, and put the cleaned text in a new column
df['clean_text'] = df['text'].map(clean_data)
# Remove duplicate texts AFTER removing @'s, web addresses,  and non-word characthers
df = df.drop_duplicates(subset='text', keep="first")


# Types of Neural Networks
A list of different types of neural networks and their pros and cons for this particular project.


---



1.   Feed Forward (Simple) Neural Net
>   Easy to understand.  Faster to run.  Doesn't preserve any word order or relationships between the words in the tweets.




2.   Recurrent Neural Network (RNN)
>  Will retain some sequence information between words in the tweet. Still simple architecture, slower than feed forward. 



3. Convolutional Neural Network (CNN)
> Will look for patterns / sequences of words that are more likely to fit our pattern. 


4. Transformer
> Uses attention masks to process the data. Good for picking out context and relationships between words. 


---






#1. Feed Forward Simple Neural Net
Single layer neural net with 10 nodes

In [ ]:


#split up the data into train / test
x_train, x_val, y_train, y_val = train_test_split(df['clean_text'], df['target'] , test_size=0.2)


# Vectorize using TFI/dF
tfidf_vect = TfidfVectorizer(max_features=5000)
tfidf_vect.fit(x_train)

x_train_tfidf = tfidf_vect.transform(x_train)
x_val_tfidf = tfidf_vect.transform(x_val)


# Convert sparse matrix to dense matrix
x_train_tfidf = x_train_tfidf.toarray()
x_val_tfidf = x_val_tfidf.toarray()

# define the model
model = Sequential()
model.add(Dense(10, input_dim=5000, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# train the model
model.fit(x_train_tfidf, y_train, epochs=10, batch_size=32, validation_data=(x_val_tfidf, y_val))

# evaluate the model
loss, accuracy = model.evaluate(x_val_tfidf, y_val)
print(f'Accuracy: {accuracy*100:.2f}%')

Epoch 1/10
188/188 [==============================] - 1s 4ms/step - loss: 0.6579 - accuracy: 0.6146 - val_loss: 0.6072 - val_accuracy: 0.7115
Epoch 2/10
188/188 [==============================] - 1s 3ms/step - loss: 0.5419 - accuracy: 0.7851 - val_loss: 0.5144 - val_accuracy: 0.7948
Epoch 3/10
188/188 [==============================] - 1s 3ms/step - loss: 0.4337 - accuracy: 0.8494 - val_loss: 0.4630 - val_accuracy: 0.8081
Epoch 4/10
188/188 [==============================] - 1s 3ms/step - loss: 0.3617 - accuracy: 0.8682 - val_loss: 0.4443 - val_accuracy: 0.8075
Epoch 5/10
188/188 [==============================] - 0s 3ms/step - loss: 0.3152 - accuracy: 0.8834 - val_loss: 0.4411 - val_accuracy: 0.8075
Epoch 6/10
188/188 [==============================] - 1s 3ms/step - loss: 0.2808 - accuracy: 0.8930 - val_loss: 0.4469 - val_accuracy: 0.8028
Epoch 7/10
188/188 [==============================] - 1s 3ms/step - loss: 0.2538 - accuracy: 0.9082 - val_loss: 0.4569 - val_accuracy: 0.7968
Epoch 

#2. Recurrent Neural Net
We will use a LSTM (Long Short Term Memory) which is a type of RNN that is better able to balance the entire tweet - doesn't weight the words at the end more heavily.

In [ ]:


#split up the data into train / test
x_train, x_val, y_train, y_val = train_test_split(df['clean_text'], df['target'], test_size=0.2)

#tokenize and prepare the text data for input into the RNN layer
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(x_train)

x_train_seq = tokenizer.texts_to_sequences(x_train)
x_val_seq = tokenizer.texts_to_sequences(x_val)

max_len = 25  # maximum length of a sentence

x_train_pad = pad_sequences(x_train_seq, maxlen=max_len, padding='post', truncating='post')
x_val_pad = pad_sequences(x_val_seq, maxlen=max_len, padding='post', truncating='post')

# define the model
model = Sequential()
model.add(Embedding(5000, 128, input_length=max_len))
model.add(LSTM(128, dropout=0.5, recurrent_dropout=0.5, return_sequences=True))
model.add(LSTM(64, dropout=0.5, recurrent_dropout=0.5, return_sequences=True))
model.add(LSTM(16, dropout=0.5, recurrent_dropout=0.5))
#model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# compile the model
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

# train the model
model.fit(x_train_pad, y_train, epochs=10, batch_size=64, validation_data=(x_val_pad, y_val))

# evaluate the model
loss, accuracy = model.evaluate(x_val_pad, y_val)
print(f'Accuracy: {accuracy*100:.2f}%')


Epoch 1/10
94/94 [==============================] - 31s 246ms/step - loss: 0.6270 - accuracy: 0.6351 - val_loss: 0.5079 - val_accuracy: 0.7648
Epoch 2/10
94/94 [==============================] - 24s 257ms/step - loss: 0.4554 - accuracy: 0.8051 - val_loss: 0.4760 - val_accuracy: 0.7821
Epoch 3/10
94/94 [==============================] - 15s 158ms/step - loss: 0.3854 - accuracy: 0.8424 - val_loss: 0.5357 - val_accuracy: 0.7801
Epoch 4/10
94/94 [==============================] - 15s 155ms/step - loss: 0.3465 - accuracy: 0.8639 - val_loss: 0.4962 - val_accuracy: 0.7888
Epoch 5/10
94/94 [==============================] - 15s 156ms/step - loss: 0.3285 - accuracy: 0.8722 - val_loss: 0.5266 - val_accuracy: 0.7881
Epoch 6/10
94/94 [==============================] - 15s 156ms/step - loss: 0.3077 - accuracy: 0.8830 - val_loss: 0.4570 - val_accuracy: 0.7928
Epoch 7/10
94/94 [==============================] - 16s 170ms/step - loss: 0.2883 - accuracy: 0.8914 - val_loss: 0.4934 - val_accuracy: 0.7901

#3.  CNN


In [ ]:
#split up the data into train / test
x_train, x_val, y_train, y_val = train_test_split(df['clean_text'], df['target'], test_size=0.2)

# Vectorize using TFI/dF
tfidf_vect = TfidfVectorizer(max_features=5000)
tfidf_vect.fit(x_train)

x_train_tfidf = tfidf_vect.transform(x_train)
x_val_tfidf = tfidf_vect.transform(x_val)

# Convert sparse matrix to dense matrix and reshape
x_train_tfidf = x_train_tfidf.toarray().reshape(x_train_tfidf.shape[0], x_train_tfidf.shape[1], 1)
x_val_tfidf = x_val_tfidf.toarray().reshape(x_val_tfidf.shape[0], x_val_tfidf.shape[1], 1)

# define the model
model = Sequential()
model.add(Conv1D(32, 3, activation='relu', input_shape=(5000, 1)))
model.add(Flatten())
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# train the model
model.fit(x_train_tfidf, y_train, epochs=10, batch_size=32, validation_data=(x_val_tfidf, y_val))

# evaluate the model
loss, accuracy = model.evaluate(x_val_tfidf, y_val)
print(f'Accuracy: {accuracy*100:.2f}%')


Epoch 1/10
188/188 [==============================] - 13s 66ms/step - loss: 0.6295 - accuracy: 0.6346 - val_loss: 0.5860 - val_accuracy: 0.8008
Epoch 2/10
188/188 [==============================] - 12s 63ms/step - loss: 0.4477 - accuracy: 0.8259 - val_loss: 0.5036 - val_accuracy: 0.7655
Epoch 3/10
188/188 [==============================] - 14s 75ms/step - loss: 0.3207 - accuracy: 0.8699 - val_loss: 0.5193 - val_accuracy: 0.7941
Epoch 4/10
188/188 [==============================] - 12s 63ms/step - loss: 0.2575 - accuracy: 0.9005 - val_loss: 0.5906 - val_accuracy: 0.7881
Epoch 5/10
188/188 [==============================] - 12s 62ms/step - loss: 0.2161 - accuracy: 0.9164 - val_loss: 0.6302 - val_accuracy: 0.7728
Epoch 6/10
188/188 [==============================] - 18s 96ms/step - loss: 0.1829 - accuracy: 0.9314 - val_loss: 0.7073 - val_accuracy: 0.7708
Epoch 7/10
188/188 [==============================] - 12s 63ms/step - loss: 0.1575 - accuracy: 0.9437 - val_loss: 0.7812 - val_accuracy:

#4. Transformer Model
Transformer models can be computationally intensive, so we are going to do the following to make it run on google colab:
1. Use DistilBert - a version of Bert but with fewer parameters
2. Use a small batch size
3. Use mixed-precision training.  This is a technique where the model is trained using half-precision floating-point numbers instead of the usual single-precision numbers.
4. Use gradient accumulation: Instead of updating the model weights after every batch, you can accumulate the gradients over several batches and update the weights once.
5. Use Data loading. 

In [ ]:
import tensorflow as tf
from transformers import DistilBertTokenizer, TFDistilBertModel
from sklearn.model_selection import train_test_split
import numpy as np

# Split data into train and validation sets
x_train, x_val, y_train, y_val = train_test_split(df['clean_text'], df['target'], test_size=0.2)

# Load DistilBert tokenizer and encode inputs
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
x_train_encodings = tokenizer(list(x_train), truncation=True, padding=True, max_length=128)
x_val_encodings = tokenizer(list(x_val), truncation=True, padding=True, max_length=128)

# Convert encodings to tensors
x_train_input_ids = tf.constant(x_train_encodings['input_ids'])
x_train_attention_masks = tf.constant(x_train_encodings['attention_mask'])
x_val_input_ids = tf.constant(x_val_encodings['input_ids'])
x_val_attention_masks = tf.constant(x_val_encodings['attention_mask'])
y_train = tf.constant(list(y_train))
y_val = tf.constant(list(y_val))

# Define model architecture
bert_layer = TFDistilBertModel.from_pretrained('distilbert-base-uncased')
input_ids = tf.keras.layers.Input(shape=(128,), dtype='int32')
attention_mask = tf.keras.layers.Input(shape=(128,), dtype='int32')
output = bert_layer(input_ids, attention_mask=attention_mask)[0]
output = tf.keras.layers.GlobalMaxPool1D()(output)
output = tf.keras.layers.Dropout(0.2)(output)
output = tf.keras.layers.Dense(1, activation='sigmoid')(output)
model = tf.keras.models.Model(inputs=[input_ids, attention_mask], outputs=output)

# Define training parameters
batch_size = 16
num_epochs = 3
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metric = tf.keras.metrics.BinaryAccuracy('accuracy')

# Compile the model
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

# Train the model
for epoch in range(num_epochs):
    print('Epoch:', epoch + 1)
    for i in range(0, len(x_train_input_ids), batch_size):
        input_ids_batch = x_train_input_ids[i:i+batch_size]
        attention_masks_batch = x_train_attention_masks[i:i+batch_size]
        labels_batch = y_train[i:i+batch_size]
        model.train_on_batch([input_ids_batch, attention_masks_batch], labels_batch)

    # Evaluate the model on validation set
    val_loss, val_accuracy = model.evaluate([x_val_input_ids, x_val_attention_masks], y_val, verbose=0)
    print('Validation accuracy:', val_accuracy)
    
# Evaluate the model on test set
test_loss, test_accuracy = model.evaluate([x_test_input_ids, x_test_attention_masks], y_test)
print('Test accuracy:', test_accuracy)


NameError: ignored

In [ ]:
#split up the data into train / test
train_texts, val_texts, train_labels, val_labels = train_test_split(df['clean_text'], df['target'], test_size=0.2)

# Define batch size and number of epochs
BATCH_SIZE = 16
EPOCHS = 5

# Load DistilBert tokenizer
tokenizer = transformers.DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

# Load training and validation data
train_data = tf.data.Dataset.from_tensor_slices((train_texts, train_labels)).shuffle(len(train_texts)).batch(BATCH_SIZE)
val_data = tf.data.Dataset.from_tensor_slices((val_texts, val_labels)).batch(BATCH_SIZE)

# Define the model
model = transformers.TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

# Define the optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)

# Define the loss function and metrics
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = [tf.keras.metrics.SparseCategoricalAccuracy('accuracy')]

# Enable mixed-precision training
policy = tf.keras.mixed_precision.experimental.Policy('mixed_float16')
tf.keras.mixed_precision.experimental.set_policy(policy)

# Define the training step
@tf.function
def train_step(inputs, labels):
    with tf.GradientTape() as tape:
        outputs = model(inputs)[0]
        loss = loss_fn(labels, outputs)
        scaled_loss = optimizer.get_scaled_loss(loss)
    scaled_gradients = tape.gradient(scaled_loss, model.trainable_variables)
    gradients = optimizer.get_unscaled_gradients(scaled_gradients)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss, metrics[0](labels, outputs)

# Define the evaluation step
@tf.function
def val_step(inputs, labels):
    outputs = model(inputs)[0]
    loss = loss_fn(labels, outputs)
    return loss, metrics[0](labels, outputs)

# Train the model
for epoch in range(EPOCHS):
    epoch_loss = tf.keras.metrics.Mean()
    epoch_accuracy = tf.keras.metrics.Mean()
    for i, (inputs, labels) in enumerate(train_data):
        loss, accuracy = train_step(tokenizer(inputs, truncation=True, padding=True, return_tensors='tf'), labels)
        epoch_loss(loss)
        epoch_accuracy(accuracy)
        if (i + 1) % 100 == 0:
            print(f'Epoch {epoch + 1}, Batch {i + 1}: Loss = {epoch_loss.result():.4f}, Accuracy = {epoch_accuracy.result():.4f}')
    epoch_loss.reset_states()
    epoch_accuracy.reset_states()
    for inputs, labels in val_data:
        loss, accuracy = val_step(tokenizer(inputs, truncation=True, padding=True, return_tensors='tf'), labels)
        epoch_loss(loss)
        epoch_accuracy(accuracy)
    print(f'Epoch {epoch + 1}, Validation Loss = {epoch_loss.result():.4f}, Validation Accuracy = {epoch_accuracy.result():.4f}')


Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_layer_norm', 'vocab_transform', 'vocab_projector', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'dropout_20', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

AttributeError: ignored

In [ ]:

# Define the tokenizer and encode the data
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
encoded_data_train = tokenizer.batch_encode_plus(df['clean_text'].to_list(),   #.values, 
                                                 add_special_tokens=True, 
                                                 return_attention_mask=True, 
                                                 pad_to_max_length=True, 
                                                 max_length=256, 
                                                 return_tensors='pt')
input_ids_train = encoded_data_train['input_ids']  # encoded_data_train is a dictionary-like object that contains the encoded representation of the text in the training set.  The 'input_ids' key contains a sequence of numerical values that correspond to the tokenized and encoded text in the training set. 
attention_masks_train = encoded_data_train['attention_mask']  #encoded_data_train['attention_mask'] is a list of binary masks that indicates which tokens in the input sequence should be attended to (i.e., given weight) by the model and which should be ignored. In other words, it tells the model which tokens are "real" inputs and which are just padding tokens. The length of the attention mask is the same as the length of the input sequence, and it has a value of 1 for all "real" tokens and 0 for all padding tokens.
labels_train = torch.tensor(df['target'].values)

# Split the data into train and validation sets
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids_train, labels_train, 
                                                                                    random_state=42, 
                                                                                    test_size=0.2)
train_masks, validation_masks, _, _ = train_test_split(attention_masks_train, input_ids_train,   #attention masks, which are binary masks that indicate which tokens are padding and which tokens are not. During training, the model learns to ignore the padded values by giving a weight of 0 to the attention masks, while giving a weight of 1 to the non-padded tokens.
                                                             random_state=42, test_size=0.2)


# Define the model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)  #DistilBert is like Bert but faster and simpler. 

# Define the optimizer and the loss function
optimizer = AdamW(model.parameters(), lr=1e-5)  # AdamW is a variant of the Adam optimizer that performs better with weight decay regularization. It computes adaptive learning rates for each parameter and momentum values for both the gradients and the parameters.
loss_fn = torch.nn.CrossEntropyLoss()  # cross entropy loss measure the difference between the predicted and actual probability distribution.  It is (y * log(p) + (1 - y) * log(1 - p)) (y is label, p is predicted)


scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_train_steps)

# enable mixed precision training
scaler = GradScaler()




Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier

NameError: ignored

#Data Loading
Data loaders are a way to load data in batches during training of a deep learning model. In machine learning, it is common to have large datasets that can't be loaded all at once into memory, especially when training on a GPU. A data loader allows you to load small batches of data from your dataset during training.

During training, you can iterate over the data loader to obtain batches of input data and their corresponding labels. Then you can feed the input data to your model, calculate the loss, backpropagate the gradients, and update the model parameters. You continue this process for multiple epochs until the model converges to a good solution.







In [ ]:
# Convert the data into Torch tensors
#Similar to a numpy array, but can also be used on a GPU to accelerate computations
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

# Define the batch size and create the DataLoader objects
batch_size = 16

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Train the model
epochs = 3
for epoch in range(epochs):
    print('step 1')
    for batch in train_dataloader:
        print('step 2')
        model.train()
        print('step 3')
        batch = tuple(t.to(device) for t in batch)
        print('step 4')
        inputs, masks, labels = batch
        print('step 5')
        optimizer.zero_grad()
        print('step 6')
        outputs = model(inputs, attention_mask=masks, labels=labels)
        print('step 7')
        loss = outputs[0]
        loss.backward()
        optimizer.step()
        print('step 8')
    
    # Validation
    model.eval()
    val_loss, val_accuracy = 0, 0
    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        inputs, masks, labels = batch
        with torch.no_grad():
            outputs = model(inputs, attention_mask=masks)
            loss = outputs[0]
            logits = outputs[1]
        val_loss += loss.item()
        val_accuracy += (logits.argmax(1) == labels).float().mean().item()
    val_loss /= len(validation_dataloader)
    val_accuracy /= len(validation_dataloader)
    print(f'Epoch {epoch+1}/{epochs} - loss: {val_loss:.4f} - accuracy: {val_accuracy:.4f}')

In [ ]:
#from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, AdamW
#import torch
#from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Define the tokenizer and encode the data
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
encoded_data_train = tokenizer.batch_encode_plus(df['clean_text'].to_list(),   #.values, 
                                                 add_special_tokens=True, 
                                                 return_attention_mask=True, 
                                                 pad_to_max_length=True, 
                                                 max_length=256, 
                                                 return_tensors='pt')
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df['target'].values)

# Split the data into train and validation sets
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids_train, labels_train, 
                                                                                    random_state=42, 
                                                                                    test_size=0.2)
train_masks, validation_masks, _, _ = train_test_split(attention_masks_train, input_ids_train, 
                                                       random_state=42, test_size=0.2)

# Convert the data into Torch tensors
#Similar to a numpy array, but can also be used on a GPU to accelerate computations
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

# Define the batch size and create the DataLoader objects
batch_size = 16

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

# Define the model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# Define the optimizer and the loss function
optimizer = AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# Train the model
epochs = 3
for epoch in range(epochs):
    for batch in train_dataloader:
        model.train()
        batch = tuple(t.to(device) for t in batch)
        inputs, masks, labels = batch
        optimizer.zero_grad()
        outputs = model(inputs, attention_mask=masks, labels=labels)
        loss = outputs[0]
        loss.backward()
        optimizer.step()
    
    # Validation
    model.eval()
    val_loss, val_accuracy = 0, 0
    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        inputs, masks, labels = batch
        with torch.no_grad():
            outputs = model(inputs, attention_mask=masks)
            loss = outputs[0]
            logits = outputs[1]
        val_loss += loss.item()
        val_accuracy += (logits.argmax(1) == labels).float().mean().item()
    val_loss /= len(validation_dataloader)
    val_accuracy /= len(validation_dataloader)
    print(f'Epoch {epoch+1}/{epochs} - loss: {val_loss:.4f} - accuracy: {val_accuracy:.4f}')


In [ ]:




print('reached this point')
# Train the model
epochs = 4
for epoch in range(epochs):
    print('before train')
    model.train()
    print('after train')
    optimizer.zero_grad()
    print('after train 1')
    outputs = model(train_inputs, attention_mask=train_masks, labels=train_labels)
    print('after train 2')
    loss = loss_fn(outputs[1], train_labels)
    print('after train 3')
    loss.backward()
    print('after train 4')
    optimizer.step()
    print(f'Epoch {epoch + 1}/{epochs} training loss: {loss.item()}')
    
    model.eval()
    predictions, true_labels = [], []
    with torch.no_grad():
        for batch in zip(validation_inputs, validation_masks):
            inputs = {'input_ids': batch[0].unsqueeze(0), 'attention_mask': batch[1].unsqueeze(0)}
            outputs = model(**inputs)
            logits = outputs[0]
            predicted_label = torch.argmax(logits).item()
            predictions.append(predicted_label)
        true_labels.extend(validation_labels.numpy())
        accuracy = (predictions == true_labels).mean()
        print(f'Epoch {epoch + 1}/{epochs} validation accuracy: {accuracy:.3f}')


In [ ]:
#split up the data into train / test
x_train, x_val, y_train, y_val = train_test_split(df['clean_text'], df['target'] , test_size=0.2)


# Vectorize using TFI/dF
tfidf_vect = TfidfVectorizer(max_features=5000)
tfidf_vect.fit(x_train)

x_train_tfidf = tfidf_vect.transform(x_train)
x_val_tfidf = tfidf_vect.transform(x_val)



In [ ]:
xtrain[5]

NameError: ignored

#Create a Neural Net

#Tokenization

Since we are going to use a transformer, we will tokenize the data using the Bert tokenizer.   Bert Tokenizer is designed specifically to work with the transformer architecture. It generates embeddings with contextual information, which is important for capturing the meaning of the text.

In [ ]:

# Define the file paths for the labeled data
#train_file = "train.txt"
#val_file = "val.txt"
#test_file = "test.txt"

# Load the labeled data
#with open(train_file, "r") as f:
#    train_texts = [line.strip() for line in f.readlines()]
#with open(val_file, "r") as f:
#    val_texts = [line.strip() for line in f.readlines()]
#with open(test_file, "r") as f:
#    test_texts = [line.strip() for line in f.readlines()]

# Define the labels for the data (0 or 1 in this case)
#train_labels = [int(text.split("\t")[1]) for text in train_texts]
#val_labels = [int(text.split("\t")[1]) for text in val_texts]
#test_labels = [int(text.split("\t")[1]) for text in test_texts]


#split up the data into train / test
x_train, x_val, y_train, y_val = train_test_split(df['clean_text'], df['target'] , test_size=0.2)

# Convert pandas Series to list of strings
x_train = x_train.tolist()
x_val = x_val.tolist()
y_train = y_train.tolist()
y_val = y_val.tolist()

# Load pre-trained tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Tokenize the text data and prepare it for model input
train_encodings = tokenizer(x_train, truncation=True, padding=True)
val_encodings = tokenizer(x_val, truncation=True, padding=True)


# Convert the tokenized data to PyTorch tensors
train_dataset = torch.utils.data.TensorDataset(
    torch.tensor(train_encodings["input_ids"]),
    torch.tensor(train_encodings["attention_mask"]),
    torch.tensor(y_train),
)
val_dataset = torch.utils.data.TensorDataset(
    torch.tensor(val_encodings["input_ids"]),
    torch.tensor(val_encodings["attention_mask"]),
    torch.tensor(y_val),
)


# Define the batch size and create data loaders for the training and validation sets
batch_size = 16
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)

# Define the optimizer and the loss function
optimizer = AdamW(model.parameters(), lr=5e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# Train the model
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        inputs = {"input_ids": batch[0], "attention_mask": batch[1], "labels": batch[2]}
        optimizer.zero_grad()
        outputs = model(**inputs)
        loss = loss_fn(outputs.logits, inputs["labels"])
        loss.backward()
        optimizer.step()

    # Evaluate the model on the validation set
    model.eval()
    val_preds = []
    val_labels = []
    with torch.no_grad():
        for batch in val_loader:
            inputs = {"input_ids": batch[0], "attention_mask": batch[1], "labels": batch[2]}
            outputs = model(**inputs)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1).tolist()
            val_preds += preds
            val_labels += inputs["labels"].tolist()
    val_acc = accuracy_score(val_labels, val_preds)
    val_precision, val_recall, val


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

KeyboardInterrupt: ignored

In [ ]:
from transformers import DistilBertTokenizerFast, TFDistilBertForSequenceClassification


# Split the data into training and validation sets
#train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

#split up the data into train / test
x_train, x_val, y_train, y_val = train_test_split(df['clean_text'], df['target'] , test_size=0.2)

# Load the tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

# Tokenize the input text and convert it into a TensorFlow dataset
train_dataset = tf.data.Dataset.from_tensor_slices(x_train.values, y_train.values) # .values?
val_dataset = tf.data.Dataset.from_tensor_slices(x_val.values,y_val.values)

# Map the tokenizer over the dataset to convert text to tokens
def tokenize(text, label):
    inputs = tokenizer(text, padding=True, truncation=True, max_length=128, return_tensors='tf')
    return inputs, label

train_dataset = train_dataset.map(tokenize, num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = val_dataset.map(tokenize, num_parallel_calls=tf.data.AUTOTUNE)

# Configure the dataset for performance
train_dataset = train_dataset.cache().shuffle(1000).batch(16).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.cache().batch(16).prefetch(tf.data.AUTOTUNE)

# Load the pre-trained DistilBert model
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5), loss=model.compute_loss, metrics=['accuracy'])

# Train the model
history = model.fit(train_dataset, validation_data=val_dataset, epochs=3)
 


ValueError: ignored

#DO NOT USE ANYTHING BELOW THIS


---



In [ ]:
# This is LSTM, but only one layer - accuracy is low
# This code works. 

#split up the data into train / test
x_train, x_val, y_train, y_val = train_test_split(df['clean_text'], df['target'], test_size=0.2)

#tokenize and prepare the text data for input into the RNN layer
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(x_train)

x_train_seq = tokenizer.texts_to_sequences(x_train)
x_val_seq = tokenizer.texts_to_sequences(x_val)

max_len = 100  # maximum length of a sentence

x_train_pad = pad_sequences(x_train_seq, maxlen=max_len, padding='post', truncating='post')
x_val_pad = pad_sequences(x_val_seq, maxlen=max_len, padding='post', truncating='post')

# define the model
model = Sequential()
model.add(Embedding(5000, 128, input_length=max_len))
model.add(LSTM(64))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# train the model
model.fit(x_train_pad, y_train, epochs=10, batch_size=32, validation_data=(x_val_pad, y_val))

# evaluate the model
loss, accuracy = model.evaluate(x_val_pad, y_val)
print(f'Accuracy: {accuracy*100:.2f}%')


Epoch 1/10
188/188 [==============================] - 15s 69ms/step - loss: 0.6837 - accuracy: 0.5746 - val_loss: 0.6855 - val_accuracy: 0.5623
Epoch 2/10
188/188 [==============================] - 14s 76ms/step - loss: 0.6829 - accuracy: 0.5766 - val_loss: 0.6854 - val_accuracy: 0.5623
Epoch 3/10
188/188 [==============================] - 15s 79ms/step - loss: 0.6830 - accuracy: 0.5770 - val_loss: 0.6854 - val_accuracy: 0.5623
Epoch 4/10
188/188 [==============================] - 12s 66ms/step - loss: 0.6827 - accuracy: 0.5766 - val_loss: 0.6854 - val_accuracy: 0.5623
Epoch 5/10
188/188 [==============================] - 12s 66ms/step - loss: 0.6826 - accuracy: 0.5766 - val_loss: 0.6854 - val_accuracy: 0.5623
Epoch 6/10
188/188 [==============================] - 13s 68ms/step - loss: 0.6823 - accuracy: 0.5766 - val_loss: 0.6854 - val_accuracy: 0.5623
Epoch 7/10
188/188 [==============================] - 19s 102ms/step - loss: 0.6822 - accuracy: 0.5766 - val_loss: 0.6854 - val_accuracy